In [288]:
import pandas as pd
apple = pd.read_csv("AAPL.csv")
apple.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099450,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094261,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087343,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089504,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092099,73449600


In [290]:
def flag_old_years(date_in_some_format):
    date_as_string = str(date_in_some_format)  # cast to string
    year_as_string = int(date_in_some_format[:4]) # last four characters
    if year_as_string < 1993:
        return "too_old"
    else:
        return date_as_string
    
apple['Date'] = apple['Date'].apply(flag_old_years)
apple = apple[apple['Date'] != "too_old"]
apple.index = range(len(apple))

In [294]:
import numpy as np
import math, cv2

def generate_n_day_images(df, n_days = 5, img_height = 32):
    i = 0
    open_ = [] 
    high_ = []
    low_ = []
    close_ = []
    volume_ = []
    moving_average = []

    past_n_closes = []
    for index, row in df.iterrows():
        # SMA = average closing price
        if index < n_days:
            past_n_closes.append(row.Close)
        open_.append(row.Open)
        high_.append(row.High)
        low_.append(row.Low)
        close_.append(row.Close)
        volume_.append(row.Volume)
        i += 1
        if i % n_days == 0:
            i = 0
            stock_image = generate_img_from_list(open_, high_, low_, close_, volume_, img_height, past_n_closes)
            stock_image = np.transpose(stock_image, (2, 1, 0))
            img = cv2.merge((stock_image[2], stock_image[1], stock_image[0]))
            fig_name = str(n_days) + "_days_" + "num_" + str(index - n_days + 1) + ".png"
            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(fig_name, img)
            past_n_closes = close_
            open_ = [] 
            high_ = []
            low_ = []
            close_ = []
            volume_ = []

    return 1

def generate_img_from_list(open_, high_, low_, close_, volume_, img_height, past_n_closes):
    n_days = len(open_)
    moving_averages = []
    for i in range(len(close_)):
        moving_averages.append(sum(past_n_closes[i:] + close_[:i]) / n_days)
    
    vol_height = round(img_height * (5/4)) - img_height
    max_price = max(high_ + moving_averages)
    min_price = min(low_ + moving_averages)
    max_volume = max(volume_)
    min_volume = min(volume_)
    price_range = max_price - min_price
    volume_range = max_volume - min_volume
    black_image_price = np.zeros(img_height*n_days*3*3)
    black_image_price = black_image_price.reshape((img_height, n_days*3, 3))
    black_image_volume = np.zeros(vol_height*n_days*3*3)
    black_image_volume = black_image_volume.reshape((vol_height, n_days*3, 3))

    idx = 0
    past_moving_averages = []
    moving_average_pixel_list = []
    for o, h, l, c, v, ma in zip(open_, high_, low_, close_, volume_, moving_averages):
        moving_average_pixel = round((ma - min_price) / price_range * (img_height-1))
        moving_average_pixel_list.append(moving_average_pixel)
        open_pxl = round((o - min_price) / price_range * (img_height-1))
        close_pxl = round((c - min_price) / price_range * (img_height-1))
        vertical_bar_first_pxl = round((l - min_price) / price_range * (img_height-1))
        vertical_bar_last_pxl = round((h - min_price) / price_range * (img_height-1))
        volume_pxl = round((v - min_volume) / volume_range * (vol_height-1))
        black_image_price[open_pxl, 3*idx].fill(255)
        black_image_price[moving_average_pixel, 3*idx+1].fill(255)
        for inner_idx in range(vertical_bar_last_pxl - vertical_bar_first_pxl):
            black_image_price[vertical_bar_first_pxl + inner_idx, 3*idx+1].fill(255)
        black_image_price[close_pxl, 3*idx+2].fill(255)
        for inner_idx_2 in range(volume_pxl):
            black_image_volume[inner_idx_2, 3*idx].fill(255)
        idx += 1

    past_pixel = moving_average_pixel_list[0]
    for i in range(len(moving_average_pixel_list)-1):
        current_pixel = moving_average_pixel_list[i]
        future_pixel = moving_average_pixel_list[i+1]
        first_column_pixel = round((current_pixel + past_pixel) / 2)
        last_column_pixel =  round((future_pixel + current_pixel) / 2)
        black_image_price[first_column_pixel, 3*i].fill(255)
        black_image_price[last_column_pixel, 3*i+2].fill(255)
        past_pixel = current_pixel
    full_stock_image = np.concatenate((black_image_volume, black_image_price), axis = 0)

    return full_stock_image

x = generate_n_day_images(apple)